In [5]:
import pandas as pd
import fitz
import re
import timeit
from timeit import Timer

import time

In [6]:
doc = fitz.open('H422 X-PRESS MULHACEN 21.3.2022.pdf')

In [7]:
shipper = (33, 140)
unit = (222, 290)
goods = (312, 445)
netw = (468, 502)
tare = (510, 547)
pod = (460, 500)
base_triangle = (33, 100, 547, 580)

In [8]:
total_starts = []   # lista över rektanglar för "Shipper :" på alla blad
total_stops = []    # lista över rektanglar för "Units :" på alla blad
total_height = 0.0  # loop över summan av höjderna på alla blad
total_words = []    # lista över alla dokumentets ord på alla blad
rect_pods = []     # lista över rektanglar för "Port of Discharge" på alla blad
words_in_rectangles = [] # lista för ord inom rektang*larna, utan "headers"
total_pods = [] # lista över rektanglar och namn för PODs
list_of_base_triangles = []
list_of_rectangles = []
list_of_tod_rectangles = []
final_tod_list = []
total_s = []

for page in doc:

    # starts
    starts = page.search_for("Shipper :")
    for rect in starts:
        total_starts.append(rect + (0, total_height, 0, total_height)) # lägger till på höjden för y0 och y1

    [total_s.append(x + (0, total_height, 0 , total_height)) for x in page.search_for("Shipper :")]

    # stops
    stops = page.search_for("Units :")
    for rect in stops:
        total_stops.append(rect + (0, total_height, 0, total_height))

    # total_words
    words_in_page = page.get_text("words")
    for words in words_in_page:
        total_words.append([words[0], words[1] + total_height, words[2], words[3] + total_height, words[4]])

    # rect_pods
    pods = page.search_for('Port of discharge')
    for rect in pods:      
        rect_pods.append(rect + (0, total_height + 12, 0, total_height + 16)) # lägger till på y0 och y1 för att hitta POD: y0=12.20998, y1=16.0859
    
    # words_in_rectangles
    base_triangle = (33, 100 + total_height, 547, 580 + total_height) # Utgår från basrektangel (33, 100, 547, 580) och lägger på totalhöjden
    words_in_rectangles += [word for word in total_words if fitz.Rect(word[:4]).intersects(base_triangle)] # Tar fram alla ord inom rektangeln ovan
    list_of_base_triangles.append(base_triangle)
    
    total_height += page.rect.height # lägger till totalhöjden varje loop
       

pod_names = [' '.join([words[4] for words in total_words if fitz.Rect(words[:4]).intersects(pod)]) for pod in rect_pods] # Tar fram alla POD is en lista

rectangle = lambda x: fitz.Rect(33, total_starts[x][1], 580, total_stops[x][3])

for num, starts in enumerate(total_starts):
    list_of_rectangles.append(rectangle(num))

for triangles, tod in zip(list_of_base_triangles, pod_names):
    list_of_tod_rectangles.append([triangles[0], triangles[1], triangles[2], triangles[3], tod])

for rect in list_of_rectangles:
    counter = 0
    for tod in list_of_tod_rectangles:
        if tod[3] // rect[3] > 0 and counter == 0:
            final_tod_list.append(tod[4])
            counter == 1
            break

In [10]:
def test1():

    total_starts = list()
    for page in doc:
        starts = page.search_for("Shipper :")
        for rect in starts:
            total_starts.append(rect)
    return total_starts

def test2():
    total_s = list()
    #for page in doc:
     #   total_s = [x for x in page.search_for("Shipper :")]
    #starts = page.search_for("Shipper :")
    total_s = [[x for x in page.search_for("Shipper :")] for page in doc]

    return total_s

#x_times = 10
#
#t1 = Timer("test1()", "from __main__ import test1")
#st1 = t1.timeit(number=x_times)
#print("for loop ",st1, "milliseconds")
#
#t2 = Timer("test2()", "from __main__ import test2")
#st2 = t2.timeit(number= x_times)
#print("list comp ",st2, "milliseconds")

def time_it1():
    start = time.perf_counter()

    test1()

    end = time.perf_counter()
    print(end - start)

def time_it2():
    start = time.perf_counter()

    test2()

    end = time.perf_counter()
    print(end - start)

time_it1()
time_it2()

        

0.017187499999636202
0.014318900000034773


In [27]:
total_starts

[Rect(33.599998474121094, 99.7957534790039, 66.37638092041016, 111.04881286621094),
 Rect(33.599998474121094, 311.2257385253906, 66.37638092041016, 322.47882080078125),
 Rect(33.599998474121094, 522.40576171875, 66.37638092041016, 533.6588134765625),
 Rect(33.599998474121094, 864.9357299804688, 66.37638092041016, 876.1888122558594),
 Rect(33.599998474121094, 1085.7557373046875, 66.37638092041016, 1097.0088195800781),
 Rect(33.599998474121094, 1447.3857421875, 66.37638092041016, 1458.6388244628906),
 Rect(33.599998474121094, 1687.3857421875, 66.37638092041016, 1698.6387939453125),
 Rect(33.599998474121094, 2029.675765991211, 66.37638092041016, 2040.9288330078125),
 Rect(33.599998474121094, 2260.125732421875, 66.37638092041016, 2271.3788146972656),
 Rect(33.599998474121094, 2612.0457611083984, 66.37638092041016, 2623.298812866211),
 Rect(33.599998474121094, 2804.2057495117188, 66.37638092041016, 2815.4588317871094),
 Rect(33.599998474121094, 4243.795753479004, 66.37638092041016, 4255.048

In [20]:
for page in doc:
    
    for rect in starts:
        

33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.965728759766
66.37638092041016
13327.218811035156
33.599998474121094
13315.